In [62]:
import nltk
import re
from collections import Counter
import pandas as pd
#nltk.download("movie_reviews")
from nltk.corpus import movie_reviews as mr



In [38]:
print(len(mr.words()), len(mr.sents()), len(mr.paras()))

1583820 65258 2000


In [39]:
mr.sents()

[['plot', ':', 'two', 'teen', 'couples', 'go', 'to', 'a', 'church', 'party', ',', 'drink', 'and', 'then', 'drive', '.'], ['they', 'get', 'into', 'an', 'accident', '.'], ...]

In [56]:
# put the sentences in dataframe
list_of_sentences = []
for list_of_words in mr.sents():
    sentence = ' '.join(list_of_words)
    sentence = re.sub('\s[^a-z]|^\.', '', sentence)
    list_of_sentences.append({'sentence':sentence, 'sentence_no_spaces':re.sub(' ', '', sentence)})

df=pd.DataFrame(list_of_sentences)

In [57]:
df

,sentence,sentence_no_spaces
0,plot two teen couples go to a church party dri...,plottwoteencouplesgotoachurchpartydrinkandthen...
1,they get into an accident,theygetintoanaccident
2,one of the guys dies but his girlfriend contin...,oneoftheguysdiesbuthisgirlfriendcontinuestosee...
3,what s the deal,whatsthedeal
4,watch the movie and sorta find out,watchthemovieandsortafindout
...,...,...
65253,it s a quick straight shot to the movie s end,itsaquickstraightshottothemoviesend
65254,in terms of overall quality i would compare th...,intermsofoverallqualityiwouldcomparethetrumans...
65255,both films are well made with interesting stor...,bothfilmsarewellmadewithinterestingstoriesseti...
65256,but neither film really felt like it capitaliz...,butneitherfilmreallyfeltlikeitcapitalizedonall...


In [140]:
%%time
# build a dictionay
MAX_WORD_LEN = 10
count_segments = Counter()

# count everything
for sentence in df.sentence:
    for word in sentence.split(' '):
        count_segments[word] += 1
                               

CPU times: total: 406 ms
Wall time: 405 ms


In [141]:
len(count_segments)

42961

In [142]:
count_segments.most_common(20)

[('the', 76009),
 ('a', 37917),
 ('and', 35480),
 ('of', 33847),
 ('to', 31841),
 ('is', 25144),
 ('in', 21420),
 ('s', 18416),
 ('it', 16074),
 ('that', 15887),
 ('as', 11352),
 ('with', 10747),
 ('for', 9879),
 ('this', 9555),
 ('his', 9542),
 ('film', 9480),
 ('i', 8882),
 ('he', 8861),
 ('but', 8622),
 ('on', 7366)]

In [145]:
%%time
# make a list of frequence enough words
bag_of_words = set()
for item in count_segments.elements():
    if count_segments[item] >= 5:
        bag_of_words.add(item)

CPU times: total: 188 ms
Wall time: 199 ms


In [146]:
len(bag_of_words)

14751

In [177]:
#go over sentences without spaces - look for the longest prefix
def sentence_segmentation(sentence):
    new_sentence = ''
    from_index = 0
    prefix_len=MAX_WORD_LEN
    while from_index < len(sentence):
        to_index = min(from_index+prefix_len, len(sentence))
        if sentence[from_index:to_index] in bag_of_words:
            new_sentence += sentence[from_index:to_index] + ' '
            from_index=to_index
            prefix_len=MAX_WORD_LEN
        else:
            prefix_len -= 1
            if prefix_len==0:
                prefix_len=MAX_WORD_LEN
                from_index+=1
        
    return new_sentence.strip()



In [178]:
print(df.sentence[10])
print(df.sentence_no_spaces[10])
sentence_segmentation(df.sentence_no_spaces[10])

it starts off normal but then downshifts into this fantasy world in which you as an audience member have no idea what s going on
itstartsoffnormalbutthendownshiftsintothisfantasyworldinwhichyouasanaudiencememberhavenoideawhatsgoingon


'its t arts off normal butt hen downs hi f t sin to this fantasy world in which you as an audience member haven o idea what s going on'

In [179]:
%%time
new_sentences = []
for sentence in df.sentence_no_spaces:
    new_sentences.append(sentence_segmentation(sentence))
   
    

CPU times: total: 4.48 s
Wall time: 4.5 s


In [180]:
df['sentenced_restored'] = new_sentences

In [186]:
df.sample(20)

,sentence,sentence_no_spaces,sentenced_restored
41028,the same thing happens when you watch a horror...,thesamethinghappenswhenyouwatchahorrormovieora...,the same thing happens when you watch ah or r ...
26881,while rimbaud becomes angered at the literary ...,whilerimbaudbecomesangeredattheliteraryworldan...,while rimbaud becomes angered att he literary ...
5574,i could see myself showing this movie to frien...,icouldseemyselfshowingthismovietofriendsandget...,i could seem y self showing this movie to frie...
62412,perhaps that alone gives us hope for the future,perhapsthatalonegivesushopeforthefuture,perhaps that alone gives us hope forth e future
50497,it was very original very well shot and very n...,itwasveryoriginalverywellshotandverynostalgica...,it was very original very wells hot and very n...
56216,there ll be a boat in a little while for daddies,therellbeaboatinalittlewhilefordaddies,there ll be a boat in ali t t le while ford ad...
51411,at times i was so amazed by the computer graph...,attimesiwassoamazedbythecomputergraphicsthatyo...,att i me s i was so amazed by the computer gra...
49952,the film perfectly captures the essence of the...,thefilmperfectlycapturestheessenceofthediscope...,the film perfectly captures the essence of the...
3213,but anyway the actors do as much as they can w...,butanywaytheactorsdoasmuchastheycanwithwhatthe...,but anyway the actors do as much as they can w...
5186,in a re make of its own961 absent minded profe...,inaremakeofitsown961absentmindedprofessordisne...,in are make of its own 6 1 absent minded profe...


In [183]:
sum(df['sentenced_restored'] == df['sentence'])

8934

In [184]:
len(df)

65258

In [185]:
sum(df['sentenced_restored'] == df['sentence']) / len(df)

0.13690275521775108